In [1]:

#导入库和数据
import pyprind 
import numpy as np
import pandas as pd   
import jieba
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras import layers
from keras.layers import Embedding 
from keras import Sequential
from keras.layers import Bidirectional
from keras.layers import Dense
from keras.layers import LSTM
import datasets as tfds
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.metrics import AUC
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
#数据读取
data_first=pd.read_csv("./data/train.news.csv")
TestData_first=pd.read_csv("./data/test.feature.csv")


In [2]:
'''
#数据处理
stem = PorterStemmer()
punc = r'~`!#$%^&*()_+-=|\';":/.,?><~·！@#￥%……&*（）——+-=“：’；、。，？》《{}'


def stopwordslist(filepath):
    stop_words = [line.strip() for line in open( filepath, 'r', encoding='utf-8').readlines()]
    return stop_words

stopwords = stopwordslist('chinesestopwords.txt')

def cleaning(text):
    cutwords = list(jieba.lcut_for_search(text))
    final_cutwords = ''
    for word in cutwords:
        if word not in stopwords and punc:
            final_cutwords += word+" "
    return final_cutwords
'''
'''
t = pd.DataFrame(data_second.astype(str))
data_second["Title"] =  t["Ofiicial Account Name"] + t["Title"] + t["Report Content"]
tt = pd.DataFrame(TestData_second.astype(str))
TestData_second["Title"] = tt["Ofiicial Account Name"] + tt["Title"] + tt["Report Content"]
#数据清洗
data_second["Title"] = data_second["Title"].apply(cleaning)
TestData_second["Title"] = TestData_second["Title"].apply(cleaning)
'''
#数据处理
#提取两列：title和label，得到data_second
data_second=data_first.loc[:,['Title','label']]#10587
TestData_second=TestData_first.loc[:,['Title']]#10141
#转换为字符串
#对data_second进行分词
data_second['Title'] = data_second['Title'].apply(lambda x: ' '.join(jieba.cut(x)))
TestData_second['Title'] = TestData_second['Title'].apply(lambda x: ' '.join(jieba.cut(x)))
#对data_second中的文本进行编码映射，并将文本转换成编码
#构建词表
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n，。',)
tokenizer.fit_on_texts(data_second['Title'])
#word_size=16302
word_size=len(tokenizer.index_word)
# 将词转换为整数序列
tokens = tokenizer.texts_to_sequences(data_second['Title'])
Test_tokens=tokenizer.texts_to_sequences(TestData_second['Title'])
# 对整数序列进行补齐
maxlen = max(len(sequence) for sequence in tokens)#39
tokens_pad = sequence.pad_sequences(tokens, maxlen=58, padding='post', truncating='pre')

Test_maxlen=max(len(sequence) for sequence in Test_tokens)#58
Test_tokens_pad = sequence.pad_sequences(Test_tokens, maxlen=58, padding='post', truncating='pre')
# 构建新的DataFrame
data_third = pd.DataFrame({'Title': tokens_pad.tolist(), 'label': data_second['label']})
TestData_third=pd.DataFrame({'Title': Test_tokens_pad.tolist()})
print(data_third)
#创建tensorflow
target=data_third.pop('label')
target_tensor = tf.convert_to_tensor(target)
# 将每个numpy数组转换为tensor对象tensor_list
tensor_list = data_third['Title'].apply(tf.convert_to_tensor)
Test_tensor_list=TestData_third['Title'].apply(tf.convert_to_tensor)
# 将tensor对象堆叠成一个新的tensor对象ds_raw
ds_raw=tf.stack(tensor_list)
Test_ds_raw=tf.stack(Test_tensor_list)
#创建联合数据集dataset
dataset = tf.data.Dataset.from_tensor_slices((ds_raw, target_tensor))
Test_dataset=tf.data.Dataset.from_tensor_slices(Test_ds_raw)
#洗牌
tf.random.set_seed(1)
dataset=dataset.shuffle(10587,reshuffle_each_iteration=False)
#按7：1比例划分训练集ds_train和验证集ds_valid（9263，1324）
ds_train=dataset.take(9263)
ds_valid=dataset.skip(9263)
ds_test=Test_dataset



Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\25842\AppData\Local\Temp\jieba.cache
Loading model cost 1.038 seconds.
Prefix dict has been built successfully.


                                                   Title  label
0      [15, 5189, 6442, 5190, 65, 4134, 49, 10, 1311,...      0
1      [3513, 69, 70, 1405, 71, 432, 4, 2126, 45, 644...      0
2      [278, 202, 101, 560, 168, 12, 201, 286, 1777, ...      0
3      [4136, 193, 279, 834, 292, 506, 176, 791, 9, 6...      0
4      [133, 36, 2127, 507, 124, 77, 4, 20, 5, 154, 2...      0
...                                                  ...    ...
10582  [47, 12, 43, 92, 67, 24, 19, 18, 54, 52, 35, 0...      1
10583  [454, 147, 44, 36, 24, 20, 11119, 1, 0, 0, 0, ...      1
10584  [239, 2624, 610, 2, 11120, 9, 57, 318, 5061, 4...      1
10585  [653, 557, 18, 124, 83, 77, 4, 36, 858, 600, 3...      1
10586  [298, 4015, 445, 4930, 4016, 1291, 4017, 2420,...      1

[10587 rows x 2 columns]


In [3]:
#建立Bi-LSTM模型
embedding_dim=58
vocab_size=word_size+2  #n->n+2
tf.random.set_seed(1)

# 定义L2正则化项
l2_regularizer = tf.keras.regularizers.l2(0.001)
#模型
bi_lstm_model=tf.keras.Sequential([tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,name='embed-layer'),
                                   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,
                                                                                      name='lstm-layer',
                                                                                      return_sequences=True,
                                                                                      kernel_regularizer=l2_regularizer, 
                                                                                      recurrent_regularizer=l2_regularizer),
                                                                                      name='bi-lstm_1'),
                                   tf.keras.layers.Dropout(0.1),
                                   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, 
                                                                                      return_sequences=False,
                                                                                      kernel_regularizer=l2_regularizer, 
                                                                                      recurrent_regularizer=l2_regularizer),
                                                                                      name='bi-lstm_2'),
                                   tf.keras.layers.Dropout(0.1),
                                   tf.keras.layers.Dense(64,activation='relu'),
                                   tf.keras.layers.Dense(1,activation='sigmoid')])
bi_lstm_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 58)          945632    
                                                                 
 bi-lstm_1 (Bidirectional)   (None, None, 128)         62976     
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 bi-lstm_2 (Bidirectional)   (None, 128)               98816     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 

In [4]:
#分批
TrainData=ds_train.padded_batch(32,padded_shapes=([-1],[]))
ValidData=ds_valid.padded_batch(32,padded_shapes=([-1],[]))

#配置模型
bi_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                      loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                      metrics=['accuracy',tf.keras.metrics.AUC()])
'''
# 创建EarlyStopping回调函数
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
'''
checkpoint = ModelCheckpoint('checkpoint.hdf5', monitor='val_loss', save_best_only=True,mode='auto')
history=bi_lstm_model.fit(TrainData,validation_data=(ValidData),batch_size=32,epochs=10,callbacks=[checkpoint])
#测试，测试数据Test_dataset是tensor联合数据集,分批得到TestData

result_label = []
TestData=ds_test.batch(32)
test_result=bi_lstm_model.predict(TestData)
for i in test_result:
    if(i>=0.35):
        result_label.append(1)
    else:
        result_label.append(0)
print(test_result)#应该是0/1
print(len(result_label))

id_column = pd.DataFrame({'id': range(1, 10142)})
label_column = pd.DataFrame({'label': result_label})
submission = pd.concat([id_column,label_column], axis=1)
submission.to_csv('submit.csv', index=False)



Epoch 1/10


290/290 [==============================] - 39s 81ms/step - loss: 0.4291 - accuracy: 0.8978 - auc: 0.9363 - val_loss: 0.1496 - val_accuracy: 0.9592 - val_auc: 0.9879
Epoch 2/10


c:\Users\25842\anaconda\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


290/290 [==============================] - 21s 74ms/step - loss: 0.0946 - accuracy: 0.9777 - auc: 0.9955 - val_loss: 0.1072 - val_accuracy: 0.9721 - val_auc: 0.9915
Epoch 3/10
290/290 [==============================] - 18s 63ms/step - loss: 0.0493 - accuracy: 0.9887 - auc: 0.9985 - val_loss: 0.1060 - val_accuracy: 0.9705 - val_auc: 0.9839
Epoch 4/10
290/290 [==============================] - 16s 54ms/step - loss: 0.0358 - accuracy: 0.9921 - auc: 0.9993 - val_loss: 0.1088 - val_accuracy: 0.9690 - val_auc: 0.9859
Epoch 5/10
290/290 [==============================] - 17s 57ms/step - loss: 0.0324 - accuracy: 0.9924 - auc: 0.9991 - val_loss: 0.1054 - val_accuracy: 0.9675 - val_auc: 0.9871
Epoch 6/10
290/290 [==============================] - 18s 61ms/step - loss: 0.0282 - accuracy: 0.9926 - auc: 0.9997 - val_loss: 0.1050 - val_accuracy: 0.9698 - val_auc: 0.9856
Epoch 7/10
290/290 [==============================] - 32s 110ms/step - loss: 0.0262 - accuracy: 0.9930 - auc: 0.9993 - val_loss: 0.